## Working with LINQ

### Item 29: Prefer Iterator Methods to Returning Collections


In [1]:
public static IEnumerable<char> GenerateAlphabet() {
    var letter = 'a';
    while (letter <= 'z') {
        yield return letter;
        letter++;
    }
}

In [3]:
using System;

foreach(char c in GenerateAlphabet()) {
    Console.Write(c + " ");
}

a b c d e f g h i j k l m n o p q r s t u v w x y z 

In [4]:
var allNumbers = Enumerable.Range(0, int.MaxValue);

TThis generate-as-needed strategy highlights one other important idiom when writing iterator methods. Iterator methods create an object that knows how to generate the sequence. The code to generate the sequence executes only when a caller requests an item in the sequence. That means very little of the code executes when a generator method is called.

### Item 30: Prefer Query Syntax to Loops

In [8]:
// looping approach
var foo = new int[11];
for (var num=0; num<foo.Length; num++) {
    foo[num] = num * num;
}

foreach(var i in foo) {
    Console.Write(i + " ");
}

0 1 4 9 16 25 36 49 64 81 100 

In [18]:
// query syntax
var foo = (from n in Enumerable.Range(0, 11) select n * n).ToArray();
Array.ForEach(foo, n => Console.Write(n + " "));

0 1 4 9 16 25 36 49 64 81 100 

We can create our own ForAll implementation...

In [ ]:
public static class Extensions { // will need to do this outside of jupyter...
    public static void ForAll<T>(this IEnumerable<T> sequence, Action<T> action) {
        foreach(T item in sequence) {
            action(item);
        }
    }
}

Change the problem to generating only those pairs where the sum of X and Y is less than 5.

In [26]:
private static IEnumerable<Tuple<int, int>> ProduceIndices() {
    for (var x = 0; x < 100; x++)
        for (var y = 0; y < 100; y++)
            if (x + y < 5)
                yield return Tuple.Create(x, y);
}

var indices = ProduceIndices();
foreach(var item in indices) {
    Console.WriteLine(item);
}

(0, 0)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(1, 0)
(1, 1)
(1, 2)
(1, 3)
(2, 0)
(2, 1)
(2, 2)
(3, 0)
(3, 1)
(4, 0)


In [27]:
private static IEnumerable<Tuple<int, int>> QueryIndices() {
    return from x in Enumerable.Range(0, 100)
            from y in Enumerable.Range(0, 100)
            where x + y < 5
            select Tuple.Create(x, y);
}

var indices = QueryIndices();
foreach(var item in indices) {
    Console.WriteLine(item);
}

(0, 0)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(1, 0)
(1, 1)
(1, 2)
(1, 3)
(2, 0)
(2, 1)
(2, 2)
(3, 0)
(3, 1)
(4, 0)


### Item 31: Create Composable APIs for Sequences
To see the benefits of writing iterator methods, let’s take a simple example and examine the translation.

In [31]:
using System;
using System.Collections.Generic;

public static void Unique(IEnumerable<int> nums) {
    var uniqueVals = new HashSet<int>();
    
    foreach(var num in nums) {
        if (!uniqueVals.Contains(num)) {
            uniqueVals.Add(num);
            Console.WriteLine(num);
        }
    }
}

In [32]:
// suppose that you instead wrote it this way...
public static IEnumerable<int> UniqueV2(IEnumerable<int> nums) {
    var uniqueVals = new HashSet<int>();
    foreach(var num in nums) {
        if (!uniqueVals.Contains(num)) {
            uniqueVals.Add(num);
            yield return num;
        }
    }
}

In [35]:
var nums = new int[] {1, 2, 2, 3, 4, 5, 5};
foreach(var num in UniqueV2(nums)) {
    Console.Write(num + " ");
}

1 2 3 4 5 

### Item 32: Decouple Iterations from Actions, Predicates, and Functions

In [ ]:
// you might write like this
static IList<int> CreateSequence(int numberOfElements, int startAt, int stepBy) {
    var collection = new IList<int>(numberOfElements);
    for (int i=0; i<numberOfElements; i++) {
        collection.Add(startAt + i*stepBy);
    }
    return collection;
}